# Front matter


_Important_: On first run, set CACHING to True

In [ ]:
CACHING = False
EXPERIMENTAL = False

## Set path

In [ ]:
PATH_YIELD_ENGINE = 'src'

def set_paths() -> None:
    """
    set correct working directory and python path when started from within PyCharm
    """
    import sys
    import os
    
    if 'cwd' not in globals():
        # noinspection PyGlobalUndefined
        global cwd
        cwd = os.path.join(os.getcwd(), os.pardir)
        os.chdir(cwd)
    
    print(f"working dir is '{os.getcwd()}'")
                             
    if PATH_YIELD_ENGINE not in sys.path:
        sys.path.insert(0, PATH_YIELD_ENGINE)
    
    print(f"added `{sys.path[0]}` to python paths")

set_paths()

## Imports

In [ ]:
import logging
import os
import pickle

import numpy as np
import pandas as pd
from lightgbm.sklearn import LGBMRegressor
from matplotlib import cm
from matplotlib.pyplot import figure
from matplotlib.ticker import ScalarFormatter
from sklearn.ensemble import RandomForestRegressor
from boruta import BorutaPy

import tests
from tests.model import make_simple_transformer
from tests.paths import TEST_DATA_CSV
from yieldengine import Sample
from yieldengine.dendrogram import DendrogramDrawer
from yieldengine.dendrogram.style import FeatMapStyle, LineStyle
from yieldengine.feature.transform import SimpleImputerDF
from yieldengine.feature.selection import BorutaDF
from yieldengine.model.inspection import ModelInspector
from yieldengine.model.pipeline import PipelineDF
from yieldengine.model.selection import Model, ModelGrid, ModelRanker
from yieldengine.model.validation import CircularCrossValidator

In [ ]:
%matplotlib inline

In [ ]:
PATH_TMP = 'tmp'
MI_PKL = os.path.join(PATH_TMP, 'model_inspector.pkl')

In [ ]:
logging.basicConfig(level=logging.DEBUG)
logging.getLogger('matplotlib').setLevel(logging.WARNING)

## Load the data file

In [ ]:
inputfile_config = tests.read_test_config(section="inputfile")

batch_df = pd.read_csv(
    filepath_or_buffer=TEST_DATA_CSV,
    delimiter=inputfile_config["delimiter"],
    header=inputfile_config["header"],
    decimal=inputfile_config["decimal"],
)

batch_df = batch_df.drop(columns=["Date", "Batch Id"])

# replace values of +/- infinite with n/a, then drop all n/a columns:
batch_df = batch_df.replace([np.inf, -np.inf], np.nan).dropna(
    axis=1, how="all"
)
batch_df.head()

In [ ]:
# define a Sample based on the test batch_file
sample = Sample(observations=batch_df, target_name="Yield")
sample

In [ ]:
# define the circular cross validator with 10 folds
circular_cv = CircularCrossValidator(test_ratio=0.2, num_folds=10)

circular_cv

In [ ]:
if not CACHING:
    boruta_selector = PipelineDF(steps=[
        ('preprocess', SimpleImputerDF(strategy='median')),
        ('boruta', BorutaDF(
            estimator=RandomForestRegressor(n_jobs=4), 
            n_estimators='auto', 
            verbose=2, 
            random_state=42))])
    selected_features = boruta_selector.fit_transform(sample.features_by_type(Sample.DTYPE_NUMERICAL), sample.target)

    sample_post_boruta = sample.select_features(
        sample.features_by_type(Sample.DTYPE_OBJECT).columns
        .union(selected_features.columns))

In [ ]:
# define a transformer step based on the sample
preprocessor = make_simple_transformer(
        impute_median_columns=sample_post_boruta.features_by_type(Sample.DTYPE_NUMERICAL).columns,
        one_hot_encode_columns=sample_post_boruta.features_by_type(Sample.DTYPE_OBJECT).columns,
)
preprocessor

sample_post_boruta.feature_names-set(selected_features.columns)

In [ ]:
if not CACHING:
    # define a Model with a preprocessing pipeline
    lgbm = ModelGrid(
                model=Model(
                    preprocessing=preprocessor, estimator=LGBMRegressor()
                ),
                estimator_parameters={
                    "max_depth": [5, 10],
                    "min_split_gain": [0.1, 0.2],
                    "num_leaves": [50, 100, 200],
                    "random_state": [42],
                },
    )
    lgbm

In [ ]:
if not CACHING:
    # define a ModelRanker
    model_ranker: ModelRanker = ModelRanker(
            grids=[lgbm], cv=circular_cv, scoring="r2"
        )

    # run the ModelRanker to retrieve a ranking
    model_ranking = model_ranker.run(sample=sample_post_boruta)
    # noinspection PyStatementEffect
    model_ranking
else:
    model_ranking = None

In [ ]:
if not CACHING:
    # retrieve the best model
    best_model = model_ranking[0]
    # noinspection PyStatementEffect
    best_model
else:
    best_model = None

In [ ]:
if not CACHING:
    # define a ModelInspector
    mi = ModelInspector(
        model=best_model.model,
        cv=circular_cv,
        sample=sample
    )

    mi.feature_dependency_matrix()

    with open(MI_PKL, 'wb') as f:
        pickle.dump(mi, f)
else:
    with open(MI_PKL, 'rb') as f:
        mi = pickle.load(f)

mi

In [ ]:
# retrieve a the shap_matrix
mi.shap_matrix().head()

In [ ]:
mi.feature_dependency_matrix().head()

In [ ]:
if EXPERIMENTAL:
    mi.plot_feature_dendrogram_scipy()

In [ ]:
plasma=cm.get_cmap(name="plasma", lut=256)

In [ ]:
fig = figure(figsize=(8,16))
ax = fig.add_subplot(111)
linkage_tree = mi.cluster_dependent_features()

drawer = DendrogramDrawer(
    linkage=linkage_tree,
    style=FeatMapStyle(ax=ax),
    title='Feature clusters'
)


drawer.draw()

In [ ]:
fig = figure(figsize=(8,16))
ax = fig.add_subplot(111)
linkage_tree = mi.cluster_dependent_features()

drawer = DendrogramDrawer(
    linkage=linkage_tree,
    style=LineStyle(ax=ax),
    title='Feature clusters'
)

drawer.draw()